In [279]:
import pandas as pd
import numpy as np

In [461]:
new_file=pd.read_csv('C:/Users/yifeichongtian/Desktop/UCL course material/data analytics/3rd report/new_business.csv')

In [462]:
new_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16273 entries, 0 to 16272
Data columns (total 12 columns):
Unnamed: 0     16273 non-null int64
business_id    16273 non-null object
categories     16273 non-null object
city           16273 non-null object
stars          16273 non-null float64
state          16273 non-null object
Noise          16273 non-null object
Wi-Fi          16273 non-null object
Price          16273 non-null float64
Kids           16273 non-null bool
Take-out       16273 non-null bool
park           16273 non-null bool
dtypes: bool(3), float64(2), int64(1), object(6)
memory usage: 1.2+ MB


## Linear Feature Selection by Pearson Correlation Coefficient

In [463]:
# import sklearn and use its linear regression model
from sklearn import linear_model

In [374]:
new_file.Noise.value_counts()

average      11472
quiet         3258
loud          1151
very_loud      392
Name: Noise, dtype: int64

In [375]:
new_file['Wi-Fi'].value_counts()

no      9566
free    6578
paid     129
Name: Wi-Fi, dtype: int64

In [376]:
new_file['park'].value_counts()

True     12595
False     3678
Name: park, dtype: int64

In [464]:
from sklearn import preprocessing
np.random.seed(12)

label_encoder=preprocessing.LabelEncoder()

new_file['Noise']=label_encoder.fit_transform(new_file['Noise'])
new_file['Wi-Fi']=label_encoder.fit_transform(new_file['Wi-Fi'])
new_file['Kids']=label_encoder.fit_transform(new_file['Kids'])
new_file['Take-out']=label_encoder.fit_transform(new_file['Take-out'])
new_file['park']=label_encoder.fit_transform(new_file['park'])

In [465]:
features=["Noise","Wi-Fi","Price","Kids","Take-out","park"]
x=new_file[features]
y=new_file["stars"]

#### *There are 6 features in total, which is not many.*

And we will use Pearson Correlation Coefficient to measure how two set of values are linearly correlated, and the result value must be in the range [-1,1]. If it is equal or nearly equal to 0, it suggests that they are not correlated, but when the value gets close to one, it exhibit positive correlation, and vice verse when it gets close to minus 1.

In [470]:
import math

# Use this function to get R value (not squared)
def computeCorrelation(x, y):
    xBar = np.mean(x)
    yBar = np.mean(y)
    SSR = 0
    varX = 0
    varY = 0
    for i in range(0 , len(x)):
        diffXXBar = x[i] - xBar
        diffYYBar = y[i] - yBar
        SSR += (diffXXBar * diffYYBar)
        varX += diffXXBar**2
        varY += diffYYBar**2
        SST = math.sqrt(varX * varY)
    return SSR / SST

# An empty array for tuples
linear_features_and_R = [] 
# An empty array for selected linear features
linear_features = []
# An array for R values
R = np.zeros(len(features))
# For every feature, calculate how much it is correlated with
# 'stars rating'
for i in range(0,len(features)):
    testX = new_file[features[i]].values
    testY = y.values
    R[i] = computeCorrelation(testX, testY)
    if abs(R[i]) > 0.05:
        linear_features.append(features[i])
        linear_features_and_R.append((features[i], R[i]))

print("Here are the selected values and their R values")
for i in range(0, len(linear_features_and_R)):
    print(linear_features_and_R[i][0],":",linear_features_and_R[i][1])

Here are the selected values and their R values
Wi-Fi : -0.0578776526475
Price : 0.0804826394613
Take-out : -0.0590775725225
park : 0.188875384523


## Mutiple linear regression

In [380]:
# We can use the linear model from sklearn package
from sklearn import linear_model

In [381]:
# Update X data frames (independent variables)
X = new_file[linear_features]
# y stays the same
y = new_file["stars"]

# A new linear regression model
regr = linear_model.LinearRegression()

# Fit X and y
regr.fit(X, y)

# Print out key parameters:
print ("coefficients: ")
print (regr.coef_)
print ("intercept: ")
print (regr.intercept_)
print ()

print("The fitted equation becomes:")
for i in range(0,len(linear_features)):
    print(round(regr.coef_[i],7),end='')
    print('*',end='')
    print('x',end='')
    print(i+1,end='')
    print(' ',end='')
    print('+',end='')
    print(' ',end='')
print('(',end='')
print(regr.intercept_,end='')
print(')')

print()
print('In which,')
for i in range(0,len(linear_features)):
    print('x',end='')
    print(i+1,end='')
    print(':',end='')
    print(linear_features[i])    

coefficients: 
[-0.05570229  0.0338357  -0.11767028  0.29015455]
intercept: 
3.40806948819

The fitted equation becomes:
-0.0557023*x1 + 0.0338357*x2 + -0.1176703*x3 + 0.2901546*x4 + (3.40806948819)

In which,
x1:Wi-Fi
x2:Price
x3:Take-out
x4:park


## Non-linear features selection by Random forest for Logistic regression

In [455]:
from sklearn.ensemble import RandomForestClassifier

In [456]:
features=["Noise","Wi-Fi","Price","Kids","Take-out","park"]
x=new_file[features]
y=new_file["stars"]

In [457]:
y.value_counts()

4.0    4794
3.5    4657
3.0    2755
4.5    1894
2.5    1307
2.0     504
5.0     178
1.5     166
1.0      18
Name: stars, dtype: int64

In [459]:
y=y>y.mean()
y.value_counts()

False    9407
True     6866
Name: stars, dtype: int64

In [445]:
rf_model=RandomForestClassifier(n_estimators=1000, max_features=2, oob_score=True)
rf_model=rf_model.fit(x,y)
rf_model

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=2, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=1000, n_jobs=1, oob_score=True, random_state=None,
            verbose=0, warm_start=False)

In [446]:
rf_model.oob_score_

0.60087261107355738

In [447]:
for features,importance in zip(features, rf_model.feature_importances_):
    print (features, ":", importance)

Noise : 0.366170891344
Wi-Fi : 0.132678173448
Price : 0.216518975017
Kids : 0.0716142026604
Take-out : 0.0692560691238
park : 0.143761688406


In [448]:
new_file["intercept"]=1
new_features=["Noise","Wi-Fi","Price","park","intercept"]
x_logit=new_file[new_features]

In [449]:
import statsmodels.api as sm
logit=sm.Logit(y,x_logit)

In [450]:
# displaying the result of logistic regression
result=logit.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.675213
         Iterations 4
                           Logit Regression Results                           
Dep. Variable:                  stars   No. Observations:                16273
Model:                          Logit   Df Residuals:                    16268
Method:                           MLE   Df Model:                            4
Date:                Tue, 14 Mar 2017   Pseudo R-squ.:                0.008361
Time:                        12:39:56   Log-Likelihood:                -10988.
converged:                       True   LL-Null:                       -11080.
                                        LLR p-value:                 5.450e-39
                 coef    std err          z      P>|z|      [95.0% Conf. Int.]
------------------------------------------------------------------------------
Noise          0.0012      0.018      0.064      0.949        -0.035     0.037
Wi-Fi         -0.1500      0.

In [451]:
print (result.params)

Noise        0.001165
Wi-Fi       -0.149993
Price        0.084494
park         0.441603
intercept   -0.709326
dtype: float64


In [453]:
# odds ratio
print (np.exp(result.params))

# odds ratio is between 0 and 1: this feature has a negative correlation with "star rating"
# odds ratio is over 1: this feature has a positive correlation with "star rating"

Noise        1.001165
Wi-Fi        0.860714
Price        1.088166
park         1.555198
intercept    0.491976
dtype: float64
